In [ ]:
%pip install --quiet -U crewai crewai_tools langchain_community crewai[google-genai]

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from crewai import Agent, Task, Crew, LLM

In [ ]:
import os

from utils import get_gemini_api_key

gemini_api_key = get_gemini_api_key()

llm = LLM(model="gemini/gemini-2.0-flash")

## Role Playing, Focus and Cooperation

In [ ]:
support_agent = Agent(
    role = "Senior Support Representative",
    goal = """
        Be the most freindly and helpful
        support representative in your team
    """,
    backstory = """
        You work at crewAI (https://crewai.com) and
        are now working on providing
        support to {customer}, a super important customer
        for your company.
        You need to make sure that you provide the best support!
        Make sure to provide full complete answers,
        and make no assumptions.
    """,
    allow_deletion = False,
    verbose = True,
    llm = llm
)

- By not setting ```allow_delegation = False ```, ```allow_delegation``` take its default value of being ```True```.
- This means the agent *can* delegate its work to another agent which is better suited to do a particular tasks.

In [ ]:
support_quality_assurance_agent = Agent(
    role = "Support Quality Assurance Specialist",
    goal = """
        Get recognition for providing the 
        best support quality assurance in your team. 
    """,
    backstory ="""
        You work at crewAI (https://crewai.com) and
        are now working with your team 
        on a request from {customer} ensuring that
        the support representative is 
        providing the best support possible.
        You need to make sure the support representative
        is providing full
        complete answers, and make no assumptions.
    """,
    verbose = True,
    llm = llm
)

## Tools, Guardrails and Memory

### Tools

In [ ]:
from crewai_tools import SerperDevTool, \
                          ScrapeWebsiteTool, \
                          WebsiteSearchTool

Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

In [ ]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [ ]:
inquiry_resolution = Task(
    description="""
    {customer} just reached out with a super important ask"
    {inquiry}
    {person} from {customer} is the one that reached out.
    Make sure to use everything you  know
    to provide the best support possible.
    You must strive to provide a complete
    and accurate response to the customer's inquiry.
    """,
    expected_output = """
    A detail, informative response to the
    customer's inquiry that addresses
    all aspects of their questions.
    The response should include references
    to everything you used to find the answer,
    Ensure the answer is complete,
    leaving no question unanswered, and maintain a helpful and friendly
    tone throughout.
    
    """,
    tools = [docs_scrape_tool],
    agent=support_agent
)

In [ ]:
quality_assurance_review = Task(
    description = """
        Review the response drafted by the Senior Support Representative for {customer}'s inquiry. 
        Ensure that the answer is comprehensive, accurate, and adheres to the 
		high-quality standards expected for customer support.
        Verify that all parts of the customer's inquiry 
        have been addressed 
		thoroughly, with a helpful and friendly tone.
        Check for references and sources used to 
        find the information, 
		ensuring the response is well-supported and 
        leaves no questions unanswered.
    """,
    expected_output="""
        A final, detailed, and informative response
        ready to be sent to the customer.
        This response should fully address the 
        customer's inquiry, incorporating all 
		relevant feedback and improvements.
		Don't be too formal, we are a chill and cool company 
	    but maintain a professional and friendly tone throughout.
    """,
    agent = support_quality_assurance_agent
)

## Creating the Crew

### Memory

In [ ]:
crew = Crew(
    agent = [support_agent, support_quality_assurance_agent],
    tasks = [inquiry_resolution, quality_assurance_review],
    verbose = True,
    memory = True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [ ]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": """
                I need to help with setting up a Crew
                and kicking it off, specifically
                how I can add memory to my crew?
                Can you provide guidance?
               """
}

result = crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)